## Data Loading in PostgreSQL Database
In this notebook, I am going to load .csv files into a PostgreSQL database. The .csv files were created with the stock_data.ipynb file. Data files are stored in the 'data' folder.

In [31]:
# Imported Libraries
import pandas as pd
import psycopg2
import os
import matplotlib
from sqlalchemy import create_engine
from tqdm import tqdm_notebook


In [32]:
# Establishing database connection

db_password = os.environ.get("POSTGRES_DB_PW") # Environmental varaible set to Postgres Password
engine = create_engine('postgresql://postgres:{}@localhost/stockdata'.format(db_password))

In [51]:
# Function to create SQL Table from a dataframe
# dataframe created using .csv files in the data folder
# Function will take the ticker as parameter

def create_stock_table(ticker):
    # Import Data as dataframe
    df = pd.read_csv('data/{}.csv'.format(ticker))
    df = df.astype({'dividends':'float', 'splits':'float'})

    # write dataframe to table in PostgreSQL
    df.to_sql('daily_prices', engine, if_exists='replace', index=False)

    query = """ALTER TABLE daily_prices
                ADD PRIMARY KEY (ticker,date);"""
    engine.execute(query)

    return "Daily Prices Table Created"

# execute function on AMZN
create_stock_table('amzn')

'Daily Prices Table Created'

In [52]:
# function will build the rest of the sql table by inserting new rows into existing table

def import_data(ticker):
    path = 'data/{}.csv'.format(ticker)
    df = pd.read_csv(path)

    insert_init = """ INSERT INTO daily_prices
                     (date, open, high, low, close, volume, dividends, splits, ticker)
                     VALUES
                    """

    vals = ",".join(["""( '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}')""".format(
        date,
        row.open,
        row.high,
        row.low,
        row.close,
        row.volume,
        row.dividends,
        row.splits,
        ticker,
    ) for date, row in df.iterrows()])
    
    insert_end = """ ON CONFLICT (ticker, date) DO UPDATE
                SET
                open = EXCLUDED.open,
                high = EXCLUDED.high,
                low = EXCLUDED.low,
                close = EXCLUDED.close,
                volume = EXCLUDED.volume,
                dividends = EXCLUDED.dividends,
                splits = EXCLUDED.splits;
                """
    
    query = insert_init + vals + insert_end

    engine.execute(query)

def stockTickers():
    tickers = [i[:-4] for i in os.listdir('data/')]

    for ticker in tickers:
        import_data(ticker)
    
    return "Ticker Loading Complete"

stockTickers()


'Ticker Loading Complete'